In [6]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [7]:
import core
import pandas as pd
import numpy as np

In [8]:
all_df = pd.read_csv("all.csv")
df_noreg = all_df[all_df["is_regularized"] == False]
df_noreg["is_big_task"] = False
df_noreg["dataset"] = df_noreg["dataset"].replace(
    {
        "cifar100_fixed": "cifar100",
        "imagenet100_subset_kaggle": "imagenet100",
    }
)

/var/folders/7p/77w4_2b565n0b8phw5c6dvtm0000gp/T/ipykernel_57596/3821391423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_noreg["is_big_task"] = False
/var/folders/7p/77w4_2b565n0b8phw5c6dvtm0000gp/T/ipykernel_57596/3821391423.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_noreg["dataset"] = df_noreg["dataset"].replace(


In [23]:
finetuning_ewc_resnet, failed_runs, not_finished_runs = core.get_df_cl("01.20", 8)
len(failed_runs)
len(not_finished_runs)
finetuning_ewc_resnet.shape


finetuning_ewc_resnet = finetuning_ewc_resnet[
    (finetuning_ewc_resnet["network"].str.contains("resnet"))
    & (finetuning_ewc_resnet["approach"].isin(["finetuning", "ewc"]))
]

1

0

(119, 86)

In [24]:
finetuning_ewc_convnext, failed_runs, not_finished_runs = core.get_df_cl("01.20", 8)
len(failed_runs)
len(not_finished_runs)
finetuning_ewc_convnext.shape

finetuning_ewc_convnext = finetuning_ewc_convnext[
    (finetuning_ewc_convnext["network"].str.contains("convnext"))
    & (finetuning_ewc_convnext["approach"].isin(["finetuning", "ewc"]))
]

1

0

(119, 86)

In [11]:
big_all_or_lwf_replay, failed_runs, not_finished_runs = core.get_df_cl("01.28", 8)
len(failed_runs)
len(not_finished_runs)
big_all_or_lwf_replay.shape

big_all = big_all_or_lwf_replay[big_all_or_lwf_replay["is_big_task"]]
lwf_replay = big_all_or_lwf_replay[~big_all_or_lwf_replay["is_big_task"]]

2

15

(139, 86)

In [12]:
reg_fetril, failed_runs, not_finished_runs = core.get_df_pycil(
    "pycil-01.28-seed-fetril"
)
len(failed_runs)
reg_fetril.shape
noreg_fetril, failed_runs, not_finished_runs = core.get_df_pycil("pycil-01.27-new")
len(failed_runs)
noreg_fetril.shape
noreg_fetril = noreg_fetril[noreg_fetril["approach"] == "fetril"]

fetril = pd.concat([reg_fetril, noreg_fetril], axis=0)

big_task_fetril, failed_runs, not_finished_runs = core.get_df_pycil(
    "pycil-01.28-big_task"
)
len(failed_runs)
big_task_fetril.shape

0

(30, 38)

38

(54, 41)

0

(24, 38)

In [27]:
pivot_table = pd.pivot_table(
    finetuning_ewc_resnet[finetuning_ewc_resnet["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                   cifar100       imagenet100   
num_tasks                       5  10 20          10 20
approach   is_regularized                              
ewc        True                  0  0  0           0  0
finetuning True                  0  0  0           0  0

In [28]:
pivot_table = pd.pivot_table(
    finetuning_ewc_convnext[
        finetuning_ewc_convnext["network"].str.contains("convnext")
    ],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                   cifar100       imagenet100   
num_tasks                       5  10 20          10 20
approach   is_regularized                              
ewc        True                  0  0  0           0  0
finetuning True                  0  0  0           0  0

In [15]:
pivot_table = pd.pivot_table(
    big_all[big_all["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    big_all[big_all["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                   cifar100 imagenet100
num_tasks                        6           6
approach   is_regularized                     
ewc        False                 0           0
           True                  0           0
finetuning False                 0           0
           True                  0           0
lwf        False                 0           0
           True                  0           0
replay     False                 0           0
           True                  0           0

dataset                   cifar100 imagenet100
num_tasks                        6           6
approach   is_regularized                     
ewc        False               0.0         0.0
           True                0.0         0.0
finetuning False               0.0         0.0
           True                0.0         0.0
lwf        False               0.0         NaN
           True                0.0         NaN
replay     False               0.0         0.0
           True                0.0         0.0

In [16]:
pivot_table = pd.pivot_table(
    lwf_replay[lwf_replay["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    lwf_replay[lwf_replay["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                 cifar100       imagenet100   
num_tasks                     5  10 20          10 20
approach is_regularized                              
lwf      True                  0  0 -2           0  0
replay   True                  0  0  0           0  0

dataset                 cifar100           imagenet100
num_tasks                     5    10   20          10
approach is_regularized                               
lwf      True                0.0  0.0  0.0         NaN
replay   True                0.0  0.0  0.0         0.0

In [17]:
pivot_table = pd.pivot_table(
    df_noreg[df_noreg["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    df_noreg[df_noreg["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                   cifar100       imagenet_subset_kaggle   
num_tasks                       5  10 20                     10 20
approach   is_regularized                                         
ewc        False                 0  0  0                      0  0
finetuning False                 0  0  0                      0  0
lwf        False                 0  0  0                      0  0
replay     False                 0  0  0                      0  0

dataset                   cifar100       imagenet_subset_kaggle   
num_tasks                       5  10 20                     10 20
approach   is_regularized                                         
ewc        False                 0  0  0                      0  0
finetuning False                 0  0  0                      0  0
lwf        False                 0  0  0                      0  0
replay     False                 0  0  0                      0  0

In [18]:
pivot_table = pd.pivot_table(
    fetril[fetril["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    fetril[fetril["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                 cifar100       imagenet100   
num_tasks                     5  10 20          10 20
approach is_regularized                              
fetril   False                 0  0  0           0  0
         True                  0  0  0           0  0

dataset                 cifar100       imagenet100   
num_tasks                     5  10 20          10 20
approach is_regularized                              
fetril   False                 0  0  0           0  0
         True                  0  0  0           0  0

In [19]:
pivot_table = pd.pivot_table(
    big_task_fetril[big_task_fetril["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    big_task_fetril[big_task_fetril["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                 cifar100 imagenet100
num_tasks                     10          10
approach is_regularized                     
fetril   False                 0           0
         True                  0           0

dataset                 cifar100 imagenet100
num_tasks                     10          10
approach is_regularized                     
fetril   False                 0           0
         True                  0           0

In [29]:
allinone = pd.concat(
    [
        finetuning_ewc_resnet,
        finetuning_ewc_convnext,
        big_all,
        lwf_replay,
        fetril,
        big_task_fetril,
        df_noreg,
    ],
    axis=0,
)

/var/folders/7p/77w4_2b565n0b8phw5c6dvtm0000gp/T/ipykernel_57596/1657574346.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  allinone = pd.concat(


In [40]:
big_all = allinone[allinone["is_big_task"]]
no_big_all = allinone[~allinone["is_big_task"]]

In [54]:
resnet = pd.pivot_table(
    no_big_all[no_big_all["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks", "seed"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
resnet
convnext = pd.pivot_table(
    no_big_all[no_big_all["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
convnext

dataset                      cifar100                                      \
num_tasks                          5                                   10   
seed                                0           1           2           0   
approach   is_regularized                                                   
ewc        False           23.69±0.00  22.44±0.00  23.16±0.00  12.19±0.00   
           True            27.14±0.00  26.83±0.00  27.66±0.00  15.70±0.00   
fetril     False           54.51±0.00  53.64±0.00  56.46±0.00  39.30±0.00   
           True            60.01±0.00  59.90±0.00  61.13±0.00  46.78±0.00   
finetuning False           20.88±0.00  20.69±0.00  20.07±0.00  11.69±0.00   
           True            23.40±0.00  23.55±0.00  23.91±0.00  14.35±0.00   
lwf        False           39.45±0.00  39.40±0.00  38.92±0.00  26.45±0.00   
           True            37.74±0.00  34.68±0.00  38.78±0.00  26.05±0.00   
replay     False           41.24±0.00  41.28±0.00  39.37±0.00  37.50±0.00   
           True            43.02±0.00  41.13±0.00  42.25±0.00  39.17±0.00   

dataset                                                                    \
num_tasks                                                  20               
seed                                1           2           0           1   
approach   is_regularized                                                   
ewc        False           12.10±0.00  12.37±0.00   6.32±0.00   1.57±0.00   
           True             8.47±0.00   9.87±0.00   2.05±0.00   9.24±0.00   
fetril     False           40.23±0.00  43.92±0.00  31.68±0.00  29.27±0.00   
           True            46.91±0.00  49.13±0.00  38.59±0.00  38.41±0.00   
finetuning False           11.45±0.00  11.49±0.00   5.25±0.00   4.20±0.00   
           True             9.92±0.00  11.63±0.00   2.52±0.00   9.01±0.00   
lwf        False           24.14±0.00  25.75±0.00  17.62±0.00     nan±nan   
           True            25.92±0.00  24.46±0.00         NaN         NaN   
replay     False           32.19±0.00  36.57±0.00  33.43±0.00  29.65±0.00   
           True            37.86±0.00  36.43±0.00  35.97±0.00  33.66±0.00   

dataset                               imagenet100                          \
num_tasks                                      10                           
seed                                2           0           1           2   
approach   is_regularized                                                   
ewc        False            6.85±0.00  13.52±0.00  13.76±0.00  13.92±0.00   
           True             6.83±0.00  21.80±0.00  20.48±0.00  20.98±0.00   
fetril     False           27.43±0.00  49.39±0.00  49.28±0.00  50.96±0.00   
           True            37.30±0.00  55.69±0.00  55.56±0.00  55.53±0.00   
finetuning False            5.65±0.00  11.56±0.00  11.64±0.00  11.36±0.00   
           True             7.63±0.00  14.80±0.00  15.20±0.00  14.78±0.00   
lwf        False           16.16±0.00  34.18±0.00  36.60±0.00  35.62±0.00   
           True            17.12±0.00  36.26±0.00  34.94±0.00  35.78±0.00   
replay     False           31.72±0.00  37.60±0.00  39.28±0.00  38.90±0.00   
           True            35.26±0.00  40.12±0.00  39.52±0.00  40.32±0.00   

dataset                                                        
num_tasks                          20                          
seed                                0           1           2  
approach   is_regularized                                      
ewc        False            6.96±0.00   7.18±0.00   6.90±0.00  
           True            11.84±0.00  11.76±0.00  11.90±0.00  
fetril     False           38.14±0.00  37.10±0.00  38.29±0.00  
           True            43.84±0.00  42.88±0.00  45.73±0.00  
finetuning False            6.38±0.00   5.78±0.00   6.08±0.00  
           True             9.84±0.00   9.06±0.00   8.72±0.00  
lwf        False           20.14±0.00  19.42±0.00  19.28±0.00  
           True            19.00±0.00  19.84±0.00  19.36±0.

dataset                      cifar100                         imagenet100  \
num_tasks                          5           10          20          10   
approach   is_regularized                                                   
ewc        False           29.57±0.03  16.92±0.55   8.87±0.53  21.65±0.70   
           True            29.05±1.21  17.02±0.69  10.26±0.07  21.60±0.70   
fetril     False           39.93±1.82  31.75±1.86  27.74±0.95  34.79±1.97   
           True            46.85±0.61  37.99±1.90  32.69±0.66  39.31±0.65   
finetuning False           26.21±0.69  15.31±0.37   8.19±0.27  17.29±0.30   
           True            24.09±0.86  13.65±0.77   8.83±0.10  16.23±0.69   
lwf        False           40.70±0.09  29.83±0.21  23.01±0.87  43.73±0.46   
           True            44.52±0.66  31.38±1.02  21.58±0.38         NaN   
replay     False           34.53±0.25  28.59±0.13  25.42±0.25  34.55±0.86   
           True            34.73±0.85  27.91±0.63  26.06±0.55  34.13±0.79   

dataset                                
num_tasks                          20  
approach   is_regularized              
ewc        False           10.81±0.39  
           True            11.85±0.20  
fetril     False           23.62±1.69  
           True            31.18±1.68  
finetuning False            7.37±0.01  
           True             8.00±0.61  
lwf        False           29.60±0.71  
           True                   NaN  
replay     False           29.78±0.74  
           True                   NaN

In [52]:
pivot_table = pd.pivot_table(
    big_all[big_all["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    big_all[big_all["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                      cifar100 imagenet100
approach   is_regularized                        
ewc        False           22.94±0.60  21.03±0.66
           True            28.94±0.59  34.05±0.41
fetril     False           64.73±0.83  73.50±0.80
           True            67.60±0.51  74.84±0.94
finetuning False           19.54±0.58  18.94±0.31
           True            24.55±0.84  24.78±0.79
lwf        False           42.16±0.80  53.53±0.62
           True            42.67±0.45  54.19±0.25
replay     False           42.69±0.55  47.09±0.89
           True            44.95±0.28  49.58±0.65

dataset                      cifar100 imagenet100
approach   is_regularized                        
ewc        False           18.09±0.62  16.68±1.22
           True            39.03±0.88  51.32±0.92
fetril     False           42.48±1.57  33.97±7.95
           True            50.26±0.53  50.99±2.65
finetuning False           19.20±0.97  14.90±1.67
           True            31.09±1.20  38.59±1.20
lwf        False           44.58±0.65         NaN
           True            44.34±0.56         NaN
replay     False           36.21±0.74  42.98±0.48
           True            36.62±0.77  44.75±0.65

In [ ]:
# allinone["dataset"] = allinone["dataset"].replace(
#         {
#             "cifar100_fixed": "cifar100",
#             "imagenet_subset_kaggle": "imagenet100",
#         }
#     )